In [352]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
from pathlib import Path
import re
import nltk
import string

plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")

pd.set_option('display.max_rows', 20)

# Setup - Load the SQL extension and connect to the Mini IMDB dataset we've prepared
db_path = Path('/Users/ryan/Downloads/biosample_basex_data_good_subset.db')

engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = sqlalchemy.inspect(engine)

query_name = """
SELECT *
FROM harmonized_wide_sel_envs
"""
harmonized_wide_sel_envs_df = pd.read_sql(query_name, engine)

aerobe_pattern = "^[Aa]erob(ic)*(-){0}"
anaerobe_pattern = "^(anaero)"
facultative_pattern = "^(facultative)"
microaerophilic_pattern = "microaerophilic" #microaerophilic already looks clean
microanaerobe_pattern = "microanaerobe" #None microanaerobe ?
obligate_aerobe_pattern = "obligate aerobe" #None obligate aerobe ?
obligate_anaerobe_pattern = "^obligate anaerobe" #already looks clean

proper_values = ["aerobe", "anaerobe", "facultative", "microaerophilic", "microanaerobe", "obligate aerobe", "obligate anaerobe"]

harmonized_wide_sel_envs_df['rel_to_oxygen_rep'] = harmonized_wide_sel_envs_df['rel_to_oxygen'].astype(str)
harmonized_wide_sel_envs_df['rel_to_oxygen_rep_status'] = (harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].isin(proper_values) == False) & (harmonized_wide_sel_envs_df['rel_to_oxygen_rep'] != "None")

harmonized_wide_sel_envs_df['rel_to_oxygen_rep_status'] = harmonized_wide_sel_envs_df['rel_to_oxygen_rep_status'].replace(to_replace = False, value="Unchanged")
harmonized_wide_sel_envs_df['rel_to_oxygen_rep_status'] = harmonized_wide_sel_envs_df['rel_to_oxygen_rep_status'].replace(to_replace = True, value="Repaired") 

harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(aerobe_pattern, regex=True), 'rel_to_oxygen_rep'] = 'aerobe'
harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(anaerobe_pattern, regex=True), 'rel_to_oxygen_rep'] = 'anaerobe'
harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(facultative_pattern, regex=True), 'rel_to_oxygen_rep'] = 'facultative'
harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(microaerophilic_pattern, regex=True), 'rel_to_oxygen_rep'] = 'microaerophilic'
harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(microanaerobe_pattern, regex=True), 'rel_to_oxygen_rep'] = 'microanaerobe'
harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(obligate_aerobe_pattern, regex=True), 'rel_to_oxygen_rep'] = 'obligate aerobe'
harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(obligate_anaerobe_pattern, regex=True), 'rel_to_oxygen_rep'] = 'obligate anaerobe'

#harmonized_wide_sel_envs_df.loc[~df["rel_to_oxygen_rep"].isin(proper_values), "rel_to_oxygen_rep"] = None
harmonized_wide_sel_envs_df['rel_to_oxygen_rep'] = np.where(harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].isin(proper_values), harmonized_wide_sel_envs_df['rel_to_oxygen_rep'], None)

harmonized_wide_sel_envs_df['air_temp'] = harmonized_wide_sel_envs_df['air_temp'].str.replace(" degree Celsius", "")
harmonized_wide_sel_envs_df['air_temp'] = harmonized_wide_sel_envs_df['air_temp'] + " degree Celsius"

/var/folders/vx/5_3c8hlx4ysbsf6gq_jw_gc80000gn/T/ipykernel_37816/4045361176.py:46: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(aerobe_pattern, regex=True), 'rel_to_oxygen_rep'] = 'aerobe'
/var/folders/vx/5_3c8hlx4ysbsf6gq_jw_gc80000gn/T/ipykernel_37816/4045361176.py:47: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(anaerobe_pattern, regex=True), 'rel_to_oxygen_rep'] = 'anaerobe'
/var/folders/vx/5_3c8hlx4ysbsf6gq_jw_gc80000gn/T/ipykernel_37816/4045361176.py:48: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  harmonized_wide_sel_envs_df.loc[harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].str.contains(facultative_pattern, regex=True), 'rel_to_oxygen_rep'] = 'facultative'

In [353]:
print("rel_to_oxygen_rep Value Counts: ")
display(harmonized_wide_sel_envs_df['rel_to_oxygen_rep'].value_counts())
print("air_temp Value Counts: ")
display(harmonized_wide_sel_envs_df['air_temp'].value_counts())
print("air_temp Unique Values: ")
display(harmonized_wide_sel_envs_df['air_temp'].unique())

rel_to_oxygen_rep Value Counts: 


aerobe               3979
obligate anaerobe      66
anaerobe               47
facultative            23
microaerophilic        11
Name: rel_to_oxygen_rep, dtype: int64

air_temp Value Counts: 


-1.9 degree Celsius     12
-1 degree Celsius       12
-3.3 degree Celsius     11
-10.8 degree Celsius    11
-7.1 degree Celsius     11
                        ..
35.2 degree Celsius      1
30.3 degree Celsius      1
28.7 degree Celsius      1
38.5 degree Celsius      1
29 degree Celsius        1
Name: air_temp, Length: 51, dtype: int64

air_temp Unique Values: 


array([nan, '-10.3 degree Celsius', '-11.6 degree Celsius',
       '-10.5 degree Celsius', '-9.7 degree Celsius',
       '-12.6 degree Celsius', '-13.1 degree Celsius',
       '-14 degree Celsius', '-11.9 degree Celsius',
       '-10.8 degree Celsius', '-8.6 degree Celsius',
       '-7.7 degree Celsius', '-4.5 degree Celsius',
       '-3.8 degree Celsius', '-2.1 degree Celsius',
       '-1.4 degree Celsius', '-3.3 degree Celsius',
       '-5.8 degree Celsius', '-7.1 degree Celsius', '-16 degree Celsius',
       '-14.2 degree Celsius', '-6 degree Celsius', '-3.9 degree Celsius',
       '-1 degree Celsius', '-0.82 degree Celsius', '-4.9 degree Celsius',
       '-2.8 degree Celsius', '-1.6 degree Celsius',
       '-14.9 degree Celsius', '-1.9 degree Celsius',
       '-6.6 degree Celsius', '-5.5 degree Celsius',
       '-2.4 degree Celsius', '-0.9 degree Celsius',
       '-1.2 degree Celsius', '-1.8 degree Celsius',
       '-4.3 degree Celsius', '-5.1 degree Celsius', '-9 degree Celsius',


# Depth Column

Depth value syntax should be {float} {unit}. preferred unit is meter

Example: 10 meter

In [354]:
#regex:=
mmRegex = r"mm"
cmRegex = r"(cm)|(centimeter)"
feetRegex = r"feet|ft"
missingRegex = r".*([Mm]issing).*|.*([Nn]one).*|.*(not).*|.*(N/*A).*"
surfaceRegex = r".*[Ss]urface.*"
dateRegex = r"[0-9][-][A-Za-z]{2,}"
mixedRegex = r".*mixed.*"
topsoilRegex = r".*[Tt]op( )*soil.*"

In [355]:
harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains(dateRegex) == True]['depth']

3868    2-Jan
3869    3-Feb
3870    4-Mar
3872    2-Jan
3873    3-Feb
        ...  
4145    3-Feb
4146    4-Mar
4148    2-Jan
4149    3-Feb
4150    4-Mar
Name: depth, Length: 213, dtype: object

In [356]:
harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains(missingRegex) == True]['depth']

/var/folders/vx/5_3c8hlx4ysbsf6gq_jw_gc80000gn/T/ipykernel_37816/574580676.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains(missingRegex) == True]['depth']


257           missing
258           missing
259           missing
260           missing
261           missing
             ...     
46162    not provided
46168    not provided
46170    not provided
46200    not provided
47507    not provided
Name: depth, Length: 305, dtype: object

Reformat all values that are considered missing as "Missing"

In [357]:
harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace(missingRegex, "Missing")
harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace(mixedRegex, "Missing")
harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace(dateRegex, "Missing")

harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains(missingRegex) == True]['depth']

/var/folders/vx/5_3c8hlx4ysbsf6gq_jw_gc80000gn/T/ipykernel_37816/2360023668.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace(missingRegex, "Missing")
/var/folders/vx/5_3c8hlx4ysbsf6gq_jw_gc80000gn/T/ipykernel_37816/2360023668.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace(mixedRegex, "Missing")
/var/folders/vx/5_3c8hlx4ysbsf6gq_jw_gc80000gn/T/ipykernel_37816/2360023668.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace(dateRegex, "Missing")
/var/folders/vx/5_3c8hlx4ysbsf6gq_jw_gc80000gn/T/ipykernel_37816/2360023668.py:5: UserWarning: This pattern has match groups. To act

257      Missing
258      Missing
259      Missing
260      Missing
261      Missing
          ...   
46162    Missing
46168    Missing
46170    Missing
46200    Missing
47507    Missing
Name: depth, Length: 575, dtype: object

In [358]:
#harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains("0 m") == True]['depth']

In [359]:
#harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains(surfaceRegex) == True]['depth']

Replace strings containing "surface" with "0 m"

In [360]:
harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace(surfaceRegex, "0 m", regex=True)

harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains(surfaceRegex) == True]['depth']

Series([], Name: depth, dtype: object)

In [361]:
#harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains("0 m") == True]['depth'].value_counts()

In [362]:
harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace(mixedRegex, "Missing", regex=True)
harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains(mixedRegex) == True]['depth'].value_counts()

Series([], Name: depth, dtype: int64)

In [363]:
harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains(topsoilRegex) == True]['depth']

/var/folders/vx/5_3c8hlx4ysbsf6gq_jw_gc80000gn/T/ipykernel_37816/3253176172.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  harmonized_wide_sel_envs_df[harmonized_wide_sel_envs_df["depth"].str.contains(topsoilRegex) == True]['depth']


2299    top soil
3236    top soil
5492     topsoil
5493     topsoil
5494     topsoil
          ...   
8241    top soil
8242    top soil
8243    top soil
8244    top soil
8245    top soil
Name: depth, Length: 148, dtype: object

Replace strings containing "top soil" with "0-0.1 m"

In [364]:
harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace(topsoilRegex, "0-0.1 m", regex=True)

Replace "Meters" with "m"

In [365]:
harmonized_wide_sel_envs_df["depth"] = harmonized_wide_sel_envs_df["depth"].str.replace("[Mm]eter[s]", "m", regex=True)